In [ ]:
from bs4 import BeautifulSoup                                       # HTML və XML-dən məlumat çıxarmaq üçün
from selenium import webdriver                                      # Brauzeri avtomatik idarə etmək üçün
from selenium.webdriver.chrome.service import Service               # ChromeDriver-i başlatmaq üçün
from selenium.webdriver.common.by import By                         # HTML elementlərini axtarmaq üçün (məs: By.CLASS_NAME)
from webdriver_manager.chrome import ChromeDriverManager            # ChromeDriver versiyasını avtomatik endirmək üçün
from selenium.webdriver.chrome.options import Options               # Chrome brauzerinə xüsusi ayarlar vermək üçün
from selenium.webdriver.support.ui import WebDriverWait             # Elementlərin yüklənməsini gözləmək üçün
from selenium.webdriver.support import expected_conditions as EC    # Müəyyən şərtlər ödənənə qədər gözləmək üçün
from selenium.common.exceptions import TimeoutException             # Vaxt aşımı xətasını idarə etmək üçün
from selenium.webdriver.common.keys import Keys                     # Klaviatura düymələrini simulyasiya etmək üçün (məs: ENTER)
import time                                                         # Kodda gecikmə vermək üçün (məs: time.sleep)
import pandas as pd                                                 # Verilənlərlə işləmək üçün (DataFrame, CSV və s.)
import re                                                           # Mətn təmizləmək, regex (müntəzəm ifadələr) ilə işləmək üçün
import pyodbc                                                       # SQL Server (və digər ODBC dəstəkləyən verilənlər bazaları) ilə bağlantı qurmaq üçün
from sqlalchemy import create_engine                                # SQLAlchemy ilə verilənlər bazasına bağlantı yaratmaq və pandas ilə işləmək üçün
import requests
import os
import base64
import numpy as np
import json
import queue
from threading import Thread
from concurrent.futures import ThreadPoolExecutor
import warnings
warnings.filterwarnings('ignore')
import undetected_chromedriver as uc
import random
import threading


#------------------------------------------------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------------------------------------------------


raw_data = queue.Queue()
cleaned_data = queue.Queue()
list_data = []

# Global seen_links set - bütün thread-lər arasında paylaşılan
global_seen_links = set()
# Thread-safe əməliyyatlar üçün lock
seen_links_lock = threading.Lock()

def scrap():
    base_urls = [
        # "https://bina.az/alqi-satqi",
        # "https://bina.az/kiraye",
        # "https://bina.az/kiraye?paid_daily=true",
        
        
        "https://bina.az/baki/alqi-satqi/menziller",
        "https://bina.az/baki/alqi-satqi/heyet-evleri",
        "https://bina.az/baki/alqi-satqi/ofisler",
        "https://bina.az/baki/alqi-satqi/qarajlar",
        "https://bina.az/baki/alqi-satqi/torpaq",
        "https://bina.az/baki/alqi-satqi/obyektler",
        "https://bina.az/baki/kiraye/menziller",
        "https://bina.az/baki/kiraye/heyet-evleri",
        "https://bina.az/baki/kiraye/ofisler",
        "https://bina.az/baki/kiraye/qarajlar",
        "https://bina.az/baki/kiraye/torpaq",
        "https://bina.az/baki/kiraye/obyektler"]

    def scrape_url(current_url):
        options = uc.ChromeOptions()
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("--disable-blink-features=AutomationControlled")
        # options.add_argument("--start-maximized")

        driver = uc.Chrome(version_main=137, options=options, headless=False)
        # driver.maximize_window()

        print(f"[Start] {current_url}")
        i = 1
        while True:
            if "paid_daily=true" in current_url:
                page_url = f"https://bina.az/kiraye?page={i}&paid_daily=true"
            else:
                page_url = f"{current_url}?page={i}"

            try:
                driver.get(page_url)
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR,"#search-page-regular-items a[href^='/item']")))
            except TimeoutException:
                print(f"Xəta: səhifə yüklənmədi → {page_url}")
                i += 1
                continue

            for _ in range(i * 30):
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)

            elements = driver.find_elements(By.CSS_SELECTOR, "#search-page-regular-items a[href^='/item']")
            all_links = [e.get_attribute("href") for e in elements if e.get_attribute("href")]
            
            # Thread-safe şəkildə yeni linkləri ayırmaq
            new_links = []
            with seen_links_lock:
                for l in all_links:
                    if l not in global_seen_links:
                        global_seen_links.add(l)
                        new_links.append(l)


            for link in new_links:
                try:
                    driver.get(link)
                    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "h1")))
                    soup = BeautifulSoup(driver.page_source, 'html.parser')
                    data = {"URL": link}

                    #--------------------------------------------------------------------------
                    # Elan ID
                    ann_id = soup.find("div", {"class": "product-actions__id"})
                    data["ListingID"] = ann_id.text.strip() if ann_id else None

                    #--------------------------------------------------------------------------
                    # Elan adi
                    announce = soup.find("h1", {"class":"product-title"})
                    data["ListingName"] = announce.text.strip() if announce else None

                    #--------------------------------------------------------------------------        
                    # Əmlak tipini tapmaq üçün breadcrumb-dan istifadə edirik
                    type_block = soup.find_all("a", {"class":"product-breadcrumbs__i-link"})
                    if len(type_block) >= 2:
                        data['SalesType'] = type_block[0].text.strip()  
                    else:
                        data['SalesType'] = None


                    #--------------------------------------------------------------------------
                    # ayliq gunluk kiraye
                    daily_rent = soup.find("span", {"class":"price-per"})
                    data['DailyRent'] = daily_rent.text.strip() if daily_rent else None


                    #--------------------------------------------------------------------------
                    # Tarix 
                    date = soup.find_all("span", {"class":"product-statistics__i-text"})
                    if len(date) >= 2:
                        data["ListingDate"] = date[1].text.strip()
                    else:
                        data["ListingDate"] = None
                    
                    #--------------------------------------------------------------------------
                    # melumat blokunda olan datalar ucun (Kategoriya, Mertebe , otaq sayi vs.)
                    # melumatlari saxlamaq ucun dictionary (key-value)
                    data["CategoryID"] = None
                    data["Floor"] = None
                    data["FieldAreaSqm"] = None
                    data["CountRoom"] = None
                    data["Title"] = None
                    data["Mortgage"] = None
                    data["Repaired"] = None
                    data["BuildingType (ForOffice)"] = None
                    data["AreaInSot"] = None

                    # melumat bloklarini tapmaq
                    info_blok = soup.find_all("div", {"class":"product-properties__i"})

                    # FƏRQLİ ADLARIN QARŞILIĞINI SAXLAYIRIQ
                    label_mapping = {
                        "Kateqoriya": "CategoryID",
                        "Mərtəbə": "Floor",
                        "Sahə": "FieldAreaSqm",
                        "Otaq sayı": "CountRoom",
                        "Çıxarış": "Title",
                        "İpoteka": "Mortgage",
                        "Təmir": "Repaired",
                        "Binanın növü": "BuildingType (ForOffice)",
                        "Torpaq sahəsi": "AreaInSot"}


                    # her blokda basliq ve ona uygun deyerin tapilmasi
                    for block in info_blok:
                        # label_tag basliqlarin oldugu kod , span_tag deyerlerin oldugu kod
                        label_tag = block.find("label", {"class":"product-properties__i-name"})
                        span_tag = block.find("span", {"class":"product-properties__i-value"})
                        
                        # Əgər həm başlıq, həm də dəyər varsa:
                        if label_tag and span_tag:
                            label = label_tag.text.strip()  # Məs: "Kateqoriya"
                            value = span_tag.text.strip()  # Məs: "Köhne tikili"
                            
                            # # Debug üçün - hansı etiketlərin olduğunu görmək üçün
                            # print(f"Tapılan etiket: '{label}' - '{value}'")
                            
                            # Etiketləri uyğunlaşdırırıq
                            if label in label_mapping:
                                data[label_mapping[label]] = value


                    #--------------------------------------------------------------------------
                    # Qiymət
                    price = soup.find("span", {"class": "price-val"})
                    data["Price"] = price.text.strip() if price else None


                    #--------------------------------------------------------------------------
                    # 1 kv.m üçün qiymet
                    kv = soup.find_all("div", {"class":"product-price__i"})
                    if len(kv) >= 2:
                        data["PricePerSqm"] = kv[1].text.strip()
                    else:
                        data["PricePerSqm"] = None

                    
                    #--------------------------------------------------------------------------
                    # Valyuta
                    currecy = soup.find("span", {"class":"price-cur"})
                    data["Currency"] = currecy.text.strip() if currecy else None


                    #--------------------------------------------------------------------------
                    # Unvan
                    address = soup.find("div", {"class":"product-map__left__address"})
                    data["Address"] = address.text.strip() if address else None

                    #--------------------------------------------------------------------------
                    # Yerləşdiyi ərazi
                    location = soup.find("div", {"class":"location"})
                    data['CityID'] = location.text.strip() if location else None

                    #--------------------------------------------------------------------------
                    # Satici (Şəxs)
                    owner = soup.find("div", {"class":"product-owner__info-name"})
                    data["SalesPerson"] = owner.text.strip() if owner else None


                    #--------------------------------------------------------------------------
                    # Satici novu
                    owner_type = soup.find("div", {"class":"product-owner__info-region"})
                    owner_type_2 = soup.find("div", {"class":"product-owner__residence-info-region"})

                    if owner_type:
                        data['SellerType'] = owner_type.text.strip()
                    elif owner_type_2:
                        data['SellerType'] = owner_type_2.text.strip()
                    else:
                        data['SellerType'] = None


                    #--------------------------------------------------------------------------
                    # Agentlik 
                    agent = soup.find("div", {"class":"product-shop__owner-name"})
                    data["Agency"] = agent.text.strip() if agent else None


                    #--------------------------------------------------------------------------
                    # Residence Adı
                    residence = soup.find("div", {"class":"product-owner__residence-info-name"})
                    data["Residence"] = residence.text.strip() if residence else None


                    #--------------------------------------------------------------------------
                    # elaqe nomresi
                    contact = soup.find("div", {"class":"product-phones__btn-value"})
                    data["Contact"] = contact.text.strip() if contact else None


                    #--------------------------------------------------------------------------
                    # Etiket
                    labels = soup.find("ul", class_="product-extras bz-d-flex bz-align-center bz-gap-15 bz-wrap-wrap")
                    data["Labels"] = [i.get_text(strip=True) for i in labels.find_all("li")] if labels else []


                    #--------------------------------------------------------------------------
                    # Açıqlama
                    description = soup.find("div", {"class":"product-description__content"})
                    data["Description"] = description.text.strip() if description else None


                    #--------------------------------------------------------------------------
                    # Koordinatlar
                    coordinates = soup.find("div", {"id": "item_map"})
                    data["Latitude"] = coordinates.get("data-lat") if coordinates else None
                    data["Longitude"] = coordinates.get("data-lng") if coordinates else None


                    #--------------------------------------------------------------------------
                    # sekil linkleri 
                    slides = soup.find_all('div', class_='product-photos__slider-top-i')
                    image_links = []

                    for slide in slides:
                        # background-image'dən link
                        span = slide.find('span', class_='product-photos__slider-top-i_background')
                        if span and 'style' in span.attrs:
                            style = span['style']
                            start = style.find("url('") + 5
                            end = style.find("')", start)
                            url = style[start:end]
                            if url:
                                image_links.append(url)

                        # img src-dən link (əgər varsa və təkrarlanmırsa)
                        img = slide.find('img')
                        if img and img.get('src') and img['src'] not in image_links:
                            image_links.append(img['src'])

                    data["Images"] = image_links if image_links else None


                    #--------------------------------------------------------------------------
                    # # Fotolar
                    # ann_id_div = soup.find('div', class_='product-actions__id')
                    # ann_id = ann_id_div.text.strip().split(':')[-1].strip()

                    # # Qovluq yaradılır
                    # image_dir = f'sekiller/{ann_id}'
                    # os.makedirs(image_dir, exist_ok=True)

                    # # Sadəcə kompüterdə şəkil kimi açılan formatlara icazə verilir
                    # sekil_formatlari = ['jpg', 'jpeg', 'png', 'webp']

                    # # Şəkilləri tap
                    # sekiller = soup.find_all('img')
                    # count = 1

                    # for img in sekiller:
                    #     src = img.get('src')
                    #     if not src:
                    #         continue

                    #     try:
                    #         if src.startswith('data:image'):
                    #             # Base64 formatında şəkil
                    #             header, encoded = src.split(',', 1)
                    #             ext = header.split('/')[1].split(';')[0].lower()

                    #             if ext not in sekil_formatlari:
                    #                 continue

                    #             image_data = base64.b64decode(encoded)
                    #         else:
                    #             # Normal linkli şəkil
                    #             ext = src.split('.')[-1].split('?')[0].lower()

                    #             if ext not in sekil_formatlari:
                    #                 continue

                    #             response = requests.get(src)
                    #             image_data = response.content

                    #         # Fayl adını .jpg kimi saxlayırıq (formatdan asılı olmayaraq)
                    #         image_path = os.path.join(image_dir, f'{ann_id}_{count}.jpg')
                    #         with open(image_path, 'wb') as f:
                    #             f.write(image_data)

                    #         # print(f'Şəkil saxlandı: {image_path}')
                    #         count += 1

                    #     except Exception as e:
                    #         print(f'X Şəkil saxlanmadı: {src}, səbəb: {e}')



                    list_data.append(data)
                    raw_data.put(pd.DataFrame([data]))
                    print(f"total: {len(list_data)})")

                except TimeoutException:
                    print(f"[Timeout] scrape edilə bilmədi → {link}")

            i += 1  
        
        # driver.quit() 

    with ThreadPoolExecutor(max_workers=len(base_urls)) as executor:
        executor.map(scrape_url, base_urls)

#------------------------------------------------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------------------------------------------------
# Cleaning

def preprocess(raw):
    df = raw.copy()

    # ListingID - integer tipinə çevrilmə
    # df['ListingID'] = df['ListingID'].astype(str).str.replace('Elanın nömrəsi:', '', regex=True)
    # df['ListingID'] = pd.to_numeric(df['ListingID'], errors='coerce')
    # df['ListingID'] = df['ListingID'].astype("Int64")  

    df['ListingID'] = df['ListingID'].astype(str).str.replace('Elanın nömrəsi:', '', regex=True)
    df['ListingID'] = pd.to_numeric(df['ListingID'], errors='coerce')
    df['ListingID'] = df['ListingID'].apply(lambda x: '101' + str(int(x)) if pd.notna(x) else x)
    df['ListingID'] = pd.to_numeric(df['ListingID'], errors='coerce').astype("Int64")


    # ListingName - string tipinə çevrilmə
    df['ListingName'] = df['ListingName'].astype("string")

    # Boolean tiplər
    df[['Title', 'Mortgage', 'Repaired']] = df[['Title', 'Mortgage', 'Repaired']].astype(bool)

    # SellerType - kategorik dəyərləri integer-ə çevirmə
    df['SellerType'] = df['SellerType'].replace({"Yaşayış kompleksi": 3, "vasitəçi (agent)": 2, "mülkiyyətçi": 1})
    df['SellerType'] = pd.to_numeric(df['SellerType'], errors='coerce')
    df['SellerType'] = df['SellerType'].astype("Int64")  

    # Contact - string tipinə çevrilmə və əvəzləmə
    df['Contact'] = df['Contact'].astype("string").str.replace("●●", "XX")

    # Description - string tipinə çevrilmə və təmizləmə
    df['Description'] = df['Description'].fillna('')
    df["Description"] = df["Description"].apply(lambda x: re.sub(r'[^\w\s,.!?()-]', '', str(x)))
    df['Description'] = df['Description'].apply(lambda x: re.sub(r'[\r\n]+', '. ', x).strip())
    df['Description'] = df['Description'].astype("string")

    # PricePerSqm - təmizləmə və integer-ə çevrilmə (NULL-larla)
    df['PricePerSqm'] = df['PricePerSqm'].astype("string").str.replace('AZN', '').str.replace(r'/m[²2]', '', regex=True).str.replace(' ', '')
    df['PricePerSqm'] = pd.to_numeric(df['PricePerSqm'], errors='coerce')
    df['PricePerSqm'] = df['PricePerSqm'].astype("Int64") 

    # CategoryID - kategorik dəyərləri integer-ə çevirmə
    df['CategoryID'] = df['CategoryID'].replace({
        "Mənzil": 1, 
        "Yeni tikili": 2, 
        "Köhnə tikili": 3, 
        "Həyət evi/bağ evi": 4,
        "Həyət evi/Bağ evi": 4,  # Həm kiçik həm böyük B variantı
        "Ofis": 5, 
        "Qaraj": 6, 
        "Torpaq": 7, 
        "Obyekt": 8
    })
    df['CategoryID'] = pd.to_numeric(df['CategoryID'], errors='coerce')
    df['CategoryID'] = df['CategoryID'].astype("Int64")  # NULL-ları saxlayır


    df["DailyRent"] = df.apply(
        lambda row: pd.NA if row["SalesType"] == "Satış" else (
            True if "/gün" in str(row["DailyRent"]).lower() or "/gun" in str(row["DailyRent"]).lower()
            else False if "/ay" in str(row["DailyRent"]).lower()
            else pd.NA
        ), axis=1
    ).astype("boolean")



    # SalesType - kategorik dəyərləri integer-ə çevirmə
    df['SalesType'] = df['SalesType'].replace({"Kirayə": 0, "Satış": 1})
    df['SalesType'] = pd.to_numeric(df['SalesType'], errors='coerce')
    df['SalesType'] = df['SalesType'].astype("Int64")  # NULL-ları saxlayır

    # FieldAreaSqm - təmizləmə və float-a çevrilmə
    df['FieldAreaSqm'] = df['FieldAreaSqm'].astype(str).str.replace('m²', '', regex=False).str.strip()
    df['FieldAreaSqm'] = pd.to_numeric(df['FieldAreaSqm'], errors='coerce')  # NULL-ları saxlayır

    # CountRoom - integer tipinə çevrilmə
    df['CountRoom'] = pd.to_numeric(df['CountRoom'], errors='coerce')
    df['CountRoom'] = df['CountRoom'].astype("Int64")  # NULL-ları saxlayır

    # Floor - string tipinə çevrilmə və mərtəbə məlumatlarının ayrılması
    df['Floor'] = df['Floor'].astype("string")
    # Floor sütunundan mərtəbə nömrəsi və ümumi mərtəbə sayı çıxarılması
    df[['FloorNumber', 'TotalFloors']] = df['Floor'].apply(lambda x: pd.Series((
        (lambda parts: (int(parts[0].strip()), int(parts[1].strip()) if len(parts) > 1 else None)
        ) if x and '/' in x else (int(x.strip()), None)
        )(str(x).split('/')) if pd.notna(x) and str(x).strip() != '' else (None, None)
    ))

    df['FloorNumber'] = df['FloorNumber'].astype("Int64")
    df['TotalFloors'] = df['TotalFloors'].astype("Int64")
    df = df.drop('Floor', axis=1)

    # AreaInSot - float tipinə çevrilmə
    df['AreaInSot'] = pd.to_numeric(df['AreaInSot'], errors='coerce')  

    # Price - təmizləmə və integer-ə çevrilmə
    df['Price'] = df['Price'].astype(str).str.replace(' ', '').str.replace(',', '')
    df['Price'] = pd.to_numeric(df['Price'], errors='coerce')
    df['Price'] = df['Price'].astype("Int64")  

    # String tipləri - NULL dəyərləri emal edilir
    df['Residence'] = df['Residence'].fillna('').astype("string")
    df['Labels'] = df['Labels'].fillna('').astype("string")
    df['Agency'] = df['Agency'].fillna('').astype("string")
    df['URL'] = df['URL'].astype("string")
    df['BuildingType (ForOffice)'] = df['BuildingType (ForOffice)'].fillna('').astype("string")
    df['Currency'] = df['Currency'].astype("string")
    df['SalesPerson'] = df['SalesPerson'].astype("string")


    df['Address'] = df['Address'].fillna('')
    df["Address"] = df["Address"].apply(lambda x: re.sub(r'[^\w\s,.!?()-]', '', str(x)))
    df['Address'] = df['Address'].apply(lambda x: re.sub(r'[\r\n]+', '. ', x).strip())
    df["Address"] = df["Address"].str.replace("User AgreementOpen in Yandex.Maps", "", regex=False)
    df["Address"] = df["Address"].str.replace("Yandex User AgreementDirections", "", regex=False)
    df["Address"] = df["Address"].str.replace("Ünvan", "", regex=False)
    df['Address'] = df['Address'].astype("string")




    # Şəhər adlarını ID-lərlə uyğunlaşdırmaq üçün xəritə (dictionary)
    city_map = {
        "Ağcabədi": 17, "Ağdam": 18, "Ağdaş": 19, "Ağstafa": 20, "Ağsu": 21,
        "Astara": 22, "Bakı": 23, "Balakən": 24, "Beyləqan": 25, "Bərdə": 26,
        "Biləsuvar": 27, "Cəlilabad": 28, "Daşkəsən": 29, "Gədəbəy": 30,
        "Gəncə": 31, "Göyçay": 32, "Göygöl": 33, "Hacıqabul": 34, "Xaçmaz": 35,
        "Xankəndi": 36, "Xırdalan": 37, "Xızı": 38, "Xudat": 39, "İmişli": 40,
        "İsmayıllı": 41, "Kürdəmir": 42, "Qax": 43, "Qazax": 44, "Qəbələ": 45,
        "Qobustan": 46, "Quba": 47, "Qusar": 48, "Lerik": 49, "Lənkəran": 50,
        "Masallı": 51, "Mingəçevir": 52, "Naxçıvan": 53, "Naxçıvan MR": 54,
        "Neftçala": 55, "Oğuz": 56, "Saatlı": 57, "Sabirabad": 58, "Salyan": 59,
        "Samux": 60, "Siyəzən": 61, "Sumqayıt": 62, "Şabran": 63, "Şamaxı": 64,
        "Şəki": 65, "Şəmkir": 66, "Şirvan": 67, "Tərtər": 68, "Tovuz": 69,
        "Ucar": 70, "Yardımlı": 71, "Yevlax": 72, "Zaqatala": 73, "Zərdab": 74
    }

    # Əgər city_map-də tapılmazsa, default olaraq Bakı (23) ilə əvəz et.
    df['CityID'] = df['CityID'].map(city_map).fillna(23).astype(int)

    # ListingDate emalı - daha güvənli
    df['ListingDate'] = df['ListingDate'].astype("string").str.replace('Yeniləndi:', '', regex=True)

    # Date və Time ayırması - xəta idarəsi ilə
    try:
        df[['Date', 'Time']] = df['ListingDate'].str.split(', ', expand=True)
        df['Date'] = pd.to_datetime(df['Date'].str.strip(), format='%d.%m.%Y', errors='coerce')
    except:
        # Əgər tarix formatında problem varsa, sadəcə boş burax
        df['Date'] = pd.NaT
        df['Time'] = ''

    df = df.drop('ListingDate', axis=1)



    #--------------------------------------------------------
    metro_mapping = {
        "20 Yanvar":1, "28 May":2, "8 Noyabr":3, "Avtovağzal":4, "Azadlıq Prospekti":5, "Bakmil":6,
        "Dərnəgül":7, "Elmlər Akademiyası":8, "Əhmədli":9, "Gənclik":10, "Həzi Aslanov":11,
        "İçəri Şəhər":12, "İnşaatçılar":13, "Koroğlu":14, "Memar Əcəmi":15, "Nəriman Nərimanov":16,
        "Nəsimi":17, "Neftçilər":18, "Nizami":19, "Qara Qarayev":20, "Sahil":21,
        "Şah İsmayıl Xətai":22, "Xalqlar Dostluğu":23, "Xocəsən":24, "Ulduz":25, "Cəfər Cabbarlı":26}

    settlement_mapping = {
        "Abşeron r.":1, "Aşağı Güzdək":2, "Atyalı":3, "Ceyranbatan":4, "Çiçək":5, "Digah":6,
        "Fatmayı":7, "Görədil":8, "Hökməli":9, "Köhnə Corat":10, "Qobu":11, "Masazır":12,
        "Mehdiabad":13, "Məmmədli":14, "Novxanı":15, "Pirəkəşkül":16, "Saray":17, "Yeni Corat":18,
        "Zuğulba":19, "Binəqədi r.":20, "2-ci Alatava":21, "28 May":22, "6-cı mikrorayon":23,
        "7-ci mikrorayon":24, "8-ci mikrorayon":25, "9-cu mikrorayon":26, "Biləcəri":27,
        "Binəqədi":28, "Xocəsən":29, "Xutor":30, "M.Ə.Rəsulzadə":31, "Sulutəpə":32,
        "Xətai r.":33, "Ağ şəhər":34, "Əhmədli":35, "Həzi Aslanov":36, "Köhnə Günəşli":37,
        "NZS":38, "Xəzər r.":39, "Binə":40, "Buzovna":41, "Dübəndi":42, "Gürgən":43,
        "Qala":44, "Mərdəkan":45, "Şağan":46, "Şimal DRES":47, "Şüvəlan":48, "Türkan":49,
        "Zirə":50, "Qaradağ r.":51, "Ələt":52, "Qızıldaş":53, "Qobustan":54, "Lökbatan":55,
        "Müşfiqabad":56, "Puta":57, "Sahil":58, "Səngəçal":59, "Şubanı":60, "Nərimanov r.":61,
        "Böyükşor":62, "Nəsimi r.":63, "1-ci mikrorayon":64, "2-ci mikrorayon":65, "3-cü mikrorayon":66,
        "4-cü mikrorayon":67, "5-ci mikrorayon":68, "Kubinka":69, "Nizami r.":70, "8-ci kilometr":71,
        "Keşlə":72, "Pirallahı r.":73, "Sabunçu r.":74, "Albalılıq":75, "Bakıxanov":76,
        "Balaxanı":77, "Bilgəh":78, "Kürdəxanı":79, "Maştağa":80, "Nardaran":81, "Pirşağı":82,
        "Ramana":83, "Sabunçu":84, "Savalan":85, "Yeni Balaxanı":86, "Yeni Ramana":87, "Zabrat":88,
        "Səbail r.":89, "20-ci sahə":90, "Badamdar":91, "Bayıl":92, "Bibiheybət":93, "Şıxov":94,
        "Suraxanı r.":95, "Bahar":96, "Bülbülə":97, "Dədə Qorqud":98, "Əmircan":99, "Günəşli":100,
        "Hövsan":101, "Qaraçuxur":102, "Massiv A":103, "Massiv B":104, "Massiv D":105,
        "Massiv G":106, "Massiv V":107, "Suraxanı":108, "Şərq":109, "Yeni Günəşli":110,
        "Yeni Suraxanı":111, "Zığ":112, "Yasamal r.":113, "Yasamal":114, "Yeni Yasamal":115, "Albalılıq":131}

    # \xa0 əvəzinə boşluq
    df['ListingName'] = df['ListingName'].str.replace('\xa0', ' ', regex=False)

    # Metro adını çıxart
    df['MetroName'] = df['ListingName'].str.extract(r',\s*([\w\s\-çğıöşüÇĞİÖŞÜƏ\.]+?)\s+m\.', expand=False).str.strip()

    # Metro mapping-ə uyğun MetroID tap
    df['MetroID'] = df['MetroName'].map(metro_mapping)

    # Settlement adını çıxart
    df['SettlementName'] = df['ListingName'].str.extract(r',\s*([\w\s\-çğıöşüÇĞİÖŞÜƏ\.]+?)\s+[rq]\.', expand=False).str.strip()

    # Mapping üçün uyğunlaşdırılmış ad sütunu (sonluqları əlavə edirik)
    df['SettlementMatch'] = df['SettlementName'].apply(lambda x: 
        f"{x} r." if f"{x} r." in settlement_mapping else (
            f"{x} q." if f"{x} q." in settlement_mapping else x
        ) if pd.notna(x) else x)

    # SettlementID tap
    df['SettlementsID'] = df['SettlementMatch'].map(settlement_mapping)

    # Təmizləmə: artıq sütunları sil
    df.drop(columns=['MetroName', 'SettlementName', 'SettlementMatch'], inplace=True)



    return df

#------------------------------------------------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------------------------------------------------


def cleaner():
    while True:
        raw = raw_data.get()        
        cleaned = preprocess(raw)        
        cleaned_data.put(cleaned)       
        raw_data.task_done()



def inserter():
    while True:
        cleaned = cleaned_data.get()
        insert_data(cleaned)
        cleaned_data.task_done()


#------------------------------------------------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------------------------------------------------
# Insert Database
def insert_data(df):    
    # Database bağlantısı
    try:
        conn = pyodbc.connect(
            "DRIVER={ODBC Driver 17 for SQL Server};"
            "SERVER=DESKTOP-KIDL0VQ\\SQLEXPRESS;" 
            "DATABASE=RealEstateDB;"
            "Trusted_Connection=yes;")
        cursor = conn.cursor()
        print("Database Connection")
    except Exception as e:
        print(f"Database Connect Xeta: {e}")
        return False

    print(f"Orijinal məlumat sayı: {len(df)}")
    df = df.drop_duplicates(subset=['ListingID'])
    print(f"Dublikatlar silindikdən sonra: {len(df)}")

    # null deyerleri ucun
    def safe_value(value):
        if pd.isna(value):
            return None
        return value

    # datalar database-de yoxlanir
    def is_listing_exists(listing_id, table_name):
        try:
            query = f"SELECT COUNT(*) FROM Dynamic.{table_name} WHERE ListingID = ?"
            cursor.execute(query, (listing_id,))
            result = cursor.fetchone()
            return result[0] > 0
        except Exception as e:
            print(f"Yoxlama xətası {table_name} - {listing_id}: {e}")
            return False

    stats = {
        'apartment': 0,
        'house': 0,
        'office': 0,
        'garage': 0,
        'land': 0,
        'commercial': 0,
        'estate_listing': 0,
        'seller_info': 0,
        'url': 0,
        'labels': 0,
        'description': 0,
        'listing_images': 0 }

    try:        
        estate_query = """
        INSERT INTO Dynamic.EstateListing (ListingID, ListingName, SalesType, DailyRent, CategoryID, SellerType, Price, PricePerSqm, Currency, CityID, Address, Date, Time, MetroID, SettlementsID, Latitude, Longitude)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """
        
        for index, row in df.iterrows():
            listing_id = safe_value(row["ListingID"])
            
            if not is_listing_exists(listing_id, "EstateListing"):
                try:
                    cursor.execute(estate_query, (listing_id,
                        safe_value(row.get("ListingName")),
                        safe_value(row.get("SalesType")),
                        safe_value(row.get("DailyRent")),
                        safe_value(row.get("CategoryID")),
                        safe_value(row.get("SellerType")),
                        safe_value(row.get("Price")),
                        safe_value(row.get("PricePerSqm")),
                        safe_value(row.get("Currency")),
                        safe_value(row.get("CityID")),
                        safe_value(row.get("Address")),
                        safe_value(row.get("Date")),
                        safe_value(row.get("Time")),
                        safe_value(row.get("MetroID")),
                        safe_value(row.get("SettlementsID")),
                        safe_value(row.get("Latitude")),
                        safe_value(row.get("Longitude"))))
                    stats['estate_listing'] += 1
                except Exception as e:
                    print(f" X EstateListing {listing_id} xətası: {e}")
        conn.commit()


        seller_query = """
        INSERT INTO Dynamic.SellerInfo (ListingID, ListingName, Contact, SellerType, Agency, Residence)
        VALUES (?, ?, ?, ?, ?, ?)
        """
        
        for index, row in df.iterrows():
            listing_id = safe_value(row["ListingID"])
            if is_listing_exists(listing_id, "EstateListing") and not is_listing_exists(listing_id, "SellerInfo"):
                try:
                    cursor.execute(seller_query, (
                        listing_id,
                        safe_value(row.get("ListingName")),
                        safe_value(row.get("Contact")),
                        safe_value(row.get("SellerType")),
                        safe_value(row.get("Agency")),
                        safe_value(row.get("Residence"))
                    ))
                    stats['seller_info'] += 1
                except Exception as e:
                    print(f"  ✗ SellerInfo {listing_id} xətası: {e}")

        description_query = """
        INSERT INTO Dynamic.Description (ListingID, Description)
        VALUES (?, ?)
        """
        
        for index, row in df.iterrows():
            listing_id = safe_value(row["ListingID"])
            if is_listing_exists(listing_id, "EstateListing") and not is_listing_exists(listing_id, "Description"):
                try:
                    cursor.execute(description_query, (
                        listing_id,
                        safe_value(row.get("Description"))
                    ))
                    stats['description'] += 1
                except Exception as e:
                    print(f"  ✗ Description {listing_id} xətası: {e}")

        conn.commit()


        if df["CategoryID"].isin([1, 2, 3]).any():
            apartment_query = """
            INSERT INTO Dynamic.Apartment (ListingID, CategoryID, FloorNumber, TotalFloors, FieldAreaSqm, CountRoom, Title, Mortgage, Repaired)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
            """
            df_apartments = df[df["CategoryID"].isin([1, 2, 3])]
            
            for index, row in df_apartments.iterrows():
                listing_id = safe_value(row["ListingID"])
                if is_listing_exists(listing_id, "EstateListing") and not is_listing_exists(listing_id, "Apartment"):
                    try:
                        cursor.execute(apartment_query, (listing_id,
                            safe_value(row["CategoryID"]),
                            safe_value(row.get("FloorNumber")),
                            safe_value(row.get("TotalFloors")),
                            safe_value(row.get("FieldAreaSqm")),
                            safe_value(row.get("CountRoom")),
                            safe_value(row.get("Title")),
                            safe_value(row.get("Mortgage")),
                            safe_value(row.get("Repaired"))))
                        stats['apartment'] += 1
                    except Exception as e:
                        print(f" X Apartment {listing_id} xətası: {e}")

        if (df["CategoryID"] == 4).any():
            house_query = """
            INSERT INTO Dynamic.House (ListingID, CategoryID, FieldAreaSqm, AreaInSot, CountRoom, Mortgage, Repaired)
            VALUES (?, ?, ?, ?, ?, ?, ?)
            """
            df_house = df[df["CategoryID"] == 4]
            
            for index, row in df_house.iterrows():
                listing_id = safe_value(row["ListingID"])
                if is_listing_exists(listing_id, "EstateListing") and not is_listing_exists(listing_id, "House"):
                    try:
                        cursor.execute(house_query, (listing_id,
                            safe_value(row["CategoryID"]),
                            safe_value(row.get("FieldAreaSqm")),
                            safe_value(row.get("AreaInSot")),
                            safe_value(row.get("CountRoom")),
                            safe_value(row.get("Mortgage")),
                            safe_value(row.get("Repaired"))))
                        stats['house'] += 1
                    except Exception as e:
                        print(f"X House {listing_id} xətası: {e}")

        if (df["CategoryID"] == 5).any():
            office_query = """
            INSERT INTO Dynamic.Office (ListingID, CategoryID, FieldAreaSqm, [BuildingType (ForOffice)], CountRoom, Title, Repaired)
            VALUES (?, ?, ?, ?, ?, ?, ?)
            """
            df_office = df[df["CategoryID"] == 5]
            
            for index, row in df_office.iterrows():
                listing_id = safe_value(row["ListingID"])
                if is_listing_exists(listing_id, "EstateListing") and not is_listing_exists(listing_id, "Office"):
                    try:
                        cursor.execute(office_query, (listing_id,
                            safe_value(row["CategoryID"]),
                            safe_value(row.get("FieldAreaSqm")),
                            safe_value(row.get("BuildingType (ForOffice)")),
                            safe_value(row.get("CountRoom")),
                            safe_value(row.get("Title")),
                            safe_value(row.get("Repaired"))))
                        stats['office'] += 1
                    except Exception as e:
                        print(f" X Office {listing_id} xətası: {e}")

        if (df["CategoryID"] == 6).any():
            garage_query = """
            INSERT INTO Dynamic.Garage (ListingID, CategoryID, FieldAreaSqm, Title)
            VALUES (?, ?, ?, ?)
            """
            df_garage = df[df["CategoryID"] == 6]
            
            for index, row in df_garage.iterrows():
                listing_id = safe_value(row["ListingID"])
                if is_listing_exists(listing_id, "EstateListing") and not is_listing_exists(listing_id, "Garage"):
                    try:
                        cursor.execute(garage_query, (listing_id,
                            safe_value(row["CategoryID"]),
                            safe_value(row.get("FieldAreaSqm")),
                            safe_value(row.get("Title"))))
                        stats['garage'] += 1
                    except Exception as e:
                        print(f" X Garage {listing_id} xətası: {e}")


        if (df["CategoryID"] == 7).any():
            land_query = """
            INSERT INTO Dynamic.Land (ListingID, CategoryID, FieldAreaSqm, Title, Mortgage)
            VALUES (?, ?, ?, ?, ?)
            """
            df_land = df[df["CategoryID"] == 7]
            
            for index, row in df_land.iterrows():
                listing_id = safe_value(row["ListingID"])
                if is_listing_exists(listing_id, "EstateListing") and not is_listing_exists(listing_id, "Land"):
                    try:
                        cursor.execute(land_query, (listing_id,
                            safe_value(row["CategoryID"]),
                            safe_value(row.get("FieldAreaSqm")),
                            safe_value(row.get("Title")),
                            safe_value(row.get("Mortgage")))) 
                        stats['land'] += 1
                    except Exception as e:
                        print(f"X Land {listing_id} xətası: {e}")

        if (df["CategoryID"] == 8).any():
            commercial_query = """
            INSERT INTO Dynamic.CommercialProperty (ListingID, CategoryID, FieldAreaSqm, Title, Mortgage, Repaired)
            VALUES (?, ?, ?, ?, ?, ?)
            """
            df_commercial = df[df["CategoryID"] == 8]
            
            for index, row in df_commercial.iterrows():
                listing_id = safe_value(row["ListingID"])
                if is_listing_exists(listing_id, "EstateListing") and not is_listing_exists(listing_id, "CommercialProperty"):
                    try:
                        cursor.execute(commercial_query, (listing_id,
                            safe_value(row["CategoryID"]),
                            safe_value(row.get("FieldAreaSqm")),
                            safe_value(row.get("Title")),
                            safe_value(row.get("Mortgage")),
                            safe_value(row.get("Repaired"))))
                        stats['commercial'] += 1
                    except Exception as e:
                        print(f"X Commercial Property {listing_id} xətası: {e}")

        conn.commit()


        # URL
        url_query = """
        INSERT INTO Dynamic.URL (ListingID, URL)
        VALUES (?, ?)
        """
        
        for index, row in df.iterrows():
            listing_id = safe_value(row["ListingID"])
            if is_listing_exists(listing_id, "EstateListing") and not is_listing_exists(listing_id, "URL"):
                try:
                    cursor.execute(url_query, (
                        listing_id,
                        safe_value(row.get("URL"))
                    ))
                    stats['url'] += 1
                except Exception as e:
                    print(f" X URL {listing_id} xətası: {e}")


        labels_query = """
        INSERT INTO Dynamic.Labels (ListingID, Labels)
        VALUES (?, ?)
        """

        def is_listing_label_exists(listing_id, label):
            check_query = "SELECT 1 FROM Dynamic.Labels WHERE ListingID = ? AND Labels = ?"
            cursor.execute(check_query, (listing_id, label))
            return cursor.fetchone() is not None

        for index, row in df.iterrows():
            listing_id = safe_value(row["ListingID"])
            if is_listing_exists(listing_id, "EstateListing"):
                labels_raw = row.get("Labels", "")
                
                labels_list = []
                if labels_raw:
                    if isinstance(labels_raw, str):
                        clean_str = str(labels_raw).replace("[", "").replace("]", "").replace("'", "").replace('"', "")
                        labels_list = [x.strip() for x in clean_str.split(",") if x.strip()]
                    elif isinstance(labels_raw, list):
                        labels_list = [str(x).strip() for x in labels_raw if str(x).strip()]
                
                for label in labels_list:
                    if not is_listing_label_exists(listing_id, label):
                        try:
                            cursor.execute(labels_query, (listing_id, label))
                            stats['labels'] += 1
                        except Exception as e:
                            print(f" X Labels {listing_id} ({label}) xətası: {e}")


        def is_image_exists(image_url):
            try:
                check_query = "SELECT COUNT(*) FROM Dynamic.ListingImages WHERE ImageURL = ?"
                cursor.execute(check_query, (image_url,))
                result = cursor.fetchone()
                return result[0] > 0
            except Exception as e:
                print(f"Image yoxlama xətası: {e}")
                return False

        listing_images_query = """
        INSERT INTO Dynamic.ListingImages (ListingID, ImageURL)
        VALUES (?, ?)
        """

        processed_images = set()

        for index, row in df.iterrows():
            listing_id = safe_value(row["ListingID"])
            if is_listing_exists(listing_id, "EstateListing"):
                image_links = row.get("Images")
                
                if image_links and listing_id:
                    if isinstance(image_links, str):
                        try:
                            import json
                            image_list = json.loads(image_links)
                        except:
                            image_list = [link.strip() for link in image_links.split(',') if link.strip()]
                    elif isinstance(image_links, list):
                        image_list = image_links
                    else:
                        image_list = []
                    
                    for image_url in image_list:
                        image_url_clean = safe_value(image_url)
                        
                        if image_url_clean and image_url_clean not in processed_images:
                            if not is_image_exists(image_url_clean):
                                try:
                                    cursor.execute(listing_images_query, (
                                        listing_id,
                                        image_url_clean
                                    ))
                                    stats['listing_images'] += 1
                                    processed_images.add(image_url_clean)
                                except Exception as e:
                                    print(f"X Image {listing_id} xətası: {e}")

        # Final commit
        conn.commit()
        print("Uğurlu")

    except Exception as e:
        print(f"Ümumi xəta baş verdi: {e}")
        try:
            conn.rollback()
            print("Rollback edildi")
        except:
            pass
        return False

    finally:
        try:
            cursor.close()
            conn.close()
            print("Database bağlantısı bağlandı")
        except:
            pass

    # Nəticələr
    print(f"   EstateListing: {stats['estate_listing']}")
    print(f"   SellerInfo:    {stats['seller_info']}")
    print(f"   Description:   {stats['description']}")
    print(f"   Apartment:     {stats['apartment']}")
    print(f"   House:         {stats['house']}")
    print(f"   Office:        {stats['office']}")
    print(f"   Garage:        {stats['garage']}")
    print(f"   Land:          {stats['land']}")
    print(f"   Commercial:    {stats['commercial']}")
    print(f"   URL:           {stats['url']}")
    print(f"   Labels:        {stats['labels']}")
    print(f"   Images:        {stats['listing_images']}")
    print(f"   TOPLAM:        {sum(stats.values())}")

    return True



if __name__ == "__main__":
    Thread(target=scrap, daemon=True).start()
    Thread(target=cleaner, daemon=True).start()
    Thread(target=inserter, daemon=True).start()

    # Prosesi aciq saxlamaq ucun
    raw_data.join()
    cleaned_data.join()
